데이콘 FIFA 선수 이적료 예측 경진대회
https://dacon.io/competitions/open/235538/overview/description

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

# 데이터 불러오기

In [2]:
train = pd.read_csv('data/fifa/FIFA_train.csv')
test = pd.read_csv('data/fifa/FIFA_test.csv')
submission  = pd.read_csv('data/fifa/submission.csv')

In [3]:
train.shape, test.shape

((8932, 12), (3828, 11))

In [4]:
train.head(2)

,id,name,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,value
0,0,L. Messi,31,south america,2021,ST,left,5.0,94,94,4.0,110500000.0
1,3,De Gea,27,europe,2020,GK,right,4.0,91,93,1.0,72000000.0


In [5]:
test.head(2)

,id,name,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves
0,1,Cristiano Ronaldo,33,europe,2022,ST,right,5.0,94,94,5.0
1,2,Neymar Jr,26,south america,2022,ST,right,5.0,92,93,5.0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8932 non-null   int64  
 1   name              8932 non-null   object 
 2   age               8932 non-null   int64  
 3   continent         8932 non-null   object 
 4   contract_until    8932 non-null   object 
 5   position          8932 non-null   object 
 6   prefer_foot       8932 non-null   object 
 7   reputation        8932 non-null   float64
 8   stat_overall      8932 non-null   int64  
 9   stat_potential    8932 non-null   int64  
 10  stat_skill_moves  8932 non-null   float64
 11  value             8932 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 837.5+ KB


In [7]:
train.describe()

,id,age,reputation,stat_overall,stat_potential,stat_skill_moves,value
count,8932.000000,8932.000000,8932.000000,8932.000000,8932.000000,8932.000000,8.932000e+03
mean,7966.775750,25.209136,1.130878,67.091133,71.997201,2.401702,2.778673e+06
std,4844.428521,4.635515,0.423792,6.854910,5.988147,0.776048,5.840982e+06
min,0.000000,16.000000,1.000000,47.000000,48.000000,1.000000,1.000000e+04
25%,3751.750000,21.000000,1.000000,63.000000,68.000000,2.000000,3.750000e+05
50%,7696.500000,25.000000,1.000000,67.000000,72.000000,2.000000,8.250000e+05
75%,12082.250000,28.000000,1.000000,72.000000,76.000000,3.000000,2.600000e+06
max,16948.000000,40.000000,5.000000,94.000000,94.000000,5.000000,1.105000e+08


In [8]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(0, 0)

In [9]:
set(train.columns) - set(test.columns)

{'value'}

In [10]:
train.nunique()

id                  8932
name                8932
age                   25
continent              5
contract_until        17
position               4
prefer_foot            2
reputation             5
stat_overall          46
stat_potential        45
stat_skill_moves       5
value                198
dtype: int64

In [11]:
test.nunique()

id                  3828
name                3828
age                   26
continent              5
contract_until        18
position               4
prefer_foot            2
reputation             5
stat_overall          47
stat_potential        43
stat_skill_moves       5
dtype: int64

# EDA 노션 참고

# 데이터 전처리

In [19]:
# 불필요한 컬럼 제거
train.drop(['id', 'name'], axis = 1, inplace = True)
test.drop(['id', 'name'], axis = 1, inplace = True)

In [20]:
train["contract_until"].value_counts()

2019            2366
2021            2308
2020            2041
2022             761
2023             506
Jun 30, 2019     501
2018             327
Dec 31, 2018      64
May 31, 2019      19
2024              12
Jan 31, 2019      10
Jun 30, 2020       9
2025               3
Jan 1, 2019        2
2026               1
May 31, 2020       1
Jan 12, 2019       1
Name: contract_until, dtype: int64

In [21]:
#contract_until 변수 int 형으로 변환
# 계약 연도만 추출
def func(string:object) -> int:
    """계약 연도만 추출하여 int로 반환"""
    string = string[-4:]
    return int(string)


train['contract_until'] = train['contract_until'].apply(func)
test['contract_until'] = test['contract_until'].apply(func)

In [ ]:
train.continent.unique()

In [ ]:
train.position.unique()

## 학습 데이터셋 나누기

In [22]:
label_name = "value"
feature_names = ['age', 'continent', 'contract_until', 'position', 'prefer_foot',
       'reputation', 'stat_overall', 'stat_potential', 'stat_skill_moves']
train[feature_names].head(2)

,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves
0,31,south america,2021,ST,left,5.0,94,94,4.0
1,27,europe,2020,GK,right,4.0,91,93,1.0


In [26]:
#원핫인코딩

X_train = pd.get_dummies(train[feature_names])
print(X_train.shape)
display(X_train.head(2))

X_test = pd.get_dummies(test[feature_names])
print(X_test.shape)
display(X_test.head(2))

y_train= train[label_name]
print(y_train.shape)
y_train.head(2)

(8932, 17)


,age,contract_until,reputation,stat_overall,stat_potential,stat_skill_moves,continent_africa,continent_asia,continent_europe,continent_oceania,continent_south america,position_DF,position_GK,position_MF,position_ST,prefer_foot_left,prefer_foot_right
0,31,2021,5.0,94,94,4.0,0,0,0,0,1,0,0,0,1,1,0
1,27,2020,4.0,91,93,1.0,0,0,1,0,0,0,1,0,0,0,1


(3828, 17)


,age,contract_until,reputation,stat_overall,stat_potential,stat_skill_moves,continent_africa,continent_asia,continent_europe,continent_oceania,continent_south america,position_DF,position_GK,position_MF,position_ST,prefer_foot_left,prefer_foot_right
0,33,2022,5.0,94,94,5.0,0,0,1,0,0,0,0,0,1,0,1
1,26,2022,5.0,92,93,5.0,0,0,0,0,1,0,0,0,1,0,1


(8932,)


0    110500000.0
1     72000000.0
Name: value, dtype: float64

In [25]:
set(X_test.columns) == set(X_train.columns)

True

## 검증용 데이터셋 나누기

In [27]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf

KFold(n_splits=5, random_state=42, shuffle=True)

# 랜덤포레스트

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor(random_state = 42)
model_rf

In [ ]:
#랜덤포레스트 점수 계산
from sklearn.model_selection import cross_val_predict
y_valid_pred = cross_val_predict(model_rf, X_train, y_train, cv=kf, n_jobs=-1)
rmse = np.square(y_train - y_valid_pred).mean() ** 0.5
rmse

In [ ]:
model_rf.fit(X_train, y_train)
y_predict = model_rf.predict(X_test)
y_predict[:5]

In [ ]:
y_predict.shape

In [ ]:
# 피처의 중요도 시각화 하기
sns.barplot(x=model_rf.feature_importances_, y=model_rf.feature_names_in_)

In [ ]:
submission['value'] = y_predict
submission.to_csv(f'data/fifa/submit_model_{rmse}.csv',index = False) 

# Gradient Boosting Model

In [ ]:
# sklearn.ensemble의 GradientBoostingRegressor
# model_gbt
from sklearn.ensemble import GradientBoostingRegressor
model_gbt = GradientBoostingRegressor(random_state = 42, n_estimators=200, learning_rate=0.9)
model_gbt

In [ ]:
# 모델을 학습시킵니다.
model_gbt.fit(X_train, y_train)

In [ ]:
y_predict = model_gbt.predict(X_test)

In [ ]:
submission['value'] = y_predict
submission.to_csv(f'data/fifa/submit_model_gbt.csv',index = False) #212위

# Catboost

In [28]:
# 코드 공유 참고
from catboost import CatBoostRegressor
cb = CatBoostRegressor(random_state = 42, silent = True, depth = 3)
# cb = CatBoostRegressor(random_state = 42)

In [29]:
#cat boost점수
from sklearn.model_selection import cross_val_predict
y_valid_pred = cross_val_predict(cb, X_train, y_train, cv=kf, n_jobs=-1)
rmse = np.square(y_train - y_valid_pred).mean() ** 0.5
rmse

612471.1589125887

In [30]:
cb.fit(X_train, y_train)
y_predict = cb.predict(X_test)

In [31]:
submission['value'] = y_predict
submission.to_csv(f'data/fifa/submit_cb4.csv',index = False) 
#catboost 147위